# Pipelime Command Line Interface

The pipelime command line interface is a powerful tool to automate data processing.
First, you can get help simply typing `pipelime`, `pipelime help`, `pipelime --help` or
even `pipelime -h`:

In [ ]:
!pipelime

The CLI is built around the concept of `Pipelime Command`, which encapsulates an
operation and makes it available both to the CLI and usual python scripting. Such
commands are dynamically loaded at runtime, so you can always run a third-party command
just by setting its full class path, eg, `my_package.my_module.MyCommand` or
`path/to/my_module.py:MyCommand`. Alternatively, let pipelime find and load your command
by setting `--module my_package.my_module` or `--module path/to/my_module.py`, then
refer to it by its pydantic title (see **#TODO REF**).

The list of available commands and sequence operators (more on this later in this doc)
can be retrieved with:

In [ ]:
!pipelime list

Now, to get help for a specific command or sequence operator, just type
`pipelime help <cmd>`, `pipelime <cmd> help`, `pipelime --help <cmd>`, etc,
eg (best viewed in a *real* terminal window):

In [ ]:
!pipelime --help clone

Also, the same help can be printed during an interactive session by explicitly calling the printer: 

In [ ]:
from pipelime.cli.utils import print_command_or_op_info

print_command_or_op_info("clone")

## Running A Command

As you can see above, the *clone* command:
* needs 3 arguments: **input** (required), **output** (required) and **grabber** (optional)
* each argument is, in fact, an *interface* encapsulating a full range of options in a tree-like structure

When you call *clone* through the pipelime cli, you can set all those options in different ways, ie:
* pydash-like key paths prefixed with "+", where the "." separates nested keys and "[]"
indexes a list, eg, `+input.folder path/to/folder`.
* a json/yaml configuration file specified as `--config path/to/cfg.yaml`. Note that command line
options update and override config file definitions.

In [ ]:
!pipelime clone +input.folder ../../tests/sample_data/datasets/underfolder_minimnist +output.folder ./clone_out +output.exists_ok=True

Likewise, the `CloneCommand` can be created and run in a python script as well:

In [ ]:
from pipelime.commands import CloneCommand
from pipelime.cli.pretty_print import print_command_outputs

cmd = CloneCommand(
    input={"folder": "../../tests/sample_data/datasets/underfolder_minimnist"},  # type: ignore
    output={"folder": "./clone_out", "exists_ok": True},  # type: ignore
)
cmd()
print_command_outputs(cmd)

## Executing A Graph Of Commands

Multiple commands can be chained ad executed as a Direct Acyclic Graph (DAG) by the *run* command (`RunCommand`):

In [ ]:
print_command_or_op_info("run")

The `nodes` attribute is a mapping of nodes, where the keys are the nodes' names and the values the actual commands to execute. As a practical example, look at the *complex_dag.yaml* file. It may seem intimidating at first, but we can easily understand the data flow by drawing it:

In [ ]:
!pipelime draw --config complex_dag.yaml

Oops! Something went wrong...
As the error message says, we need to specify some variables. To get a full list,
just audit the configuration file:

In [ ]:
!pipelime audit --config complex_dag.yaml

The `complex_params.yaml` file defines such variables, except for `params.root_folder`, which defined by the user on the command line using the special `!` prefix:

In [ ]:
!pipelime audit --config complex_dag.yaml --context complex_params.yaml !params.root_folder=./output

Now we are ready to inspect and run the computation graph:

In [ ]:
!pipelime draw --config complex_dag.yaml --context complex_params.yaml !params.root_folder=./output

In [ ]:
!pipelime run --config complex_dag.yaml --context complex_params.yaml !params.root_folder=./output

And now run again!

In [ ]:
!pipelime run --config complex_dag.yaml --context complex_params.yaml !params.root_folder=./output

Ahaa! We got an error:
```
FileExistsError: Trying to overwrite an existing dataset. Please use `exists_ok=True` to overwrite.
```
Looking at `complex_dag.yaml`, we can see that the `exists_ok` option is not set for
`nodes.sum_1.$args.output`. We can fix this by adding it on the command line (best viewed in a *real* terminal window):

In [ ]:
!pipelime run --config complex_dag.yaml --context complex_params.yaml !params.root_folder=./output "+nodes.sum_1.$args.output.exists_ok" True

## Standard Parameter Interfaces

Pipelime commands are built for best modularity and reusability, so common parameters,
such as input and output datasets, are defined in standard interfaces. Here a brief
overview.

### Input Dataset And Schema Validation

An input dataset is defined by:
* `folder`: the root folder of the dataset
* `merge_root_items`: wether shared root items should be added to each sample
* `schema`: the optional schema validation

The validation schema is mainly defined by the `schema.sample_schema` key, which is
basically a mapping from sample keys to item type, eg:
```
{
    "schema": {
        "sample_schema": {
            "image": {
                "class_path": "ImageItem",
                "is_optional": False,
            },
            "label": {
                "class_path": "TxtNumpyItem",
                "is_optional": True,
            },
        }
    }
}
```

Fine-grained validation can be performed by adding validator callables or, instead of
the dictionary above, a full-fledged pydantic model, eg:
```
$ my_schema.py
----------------------------------------------------
from pydantic import BaseModel, validator
from pipelime.items import ImageItem, TxtNumpyItem
from typing import Optional

class MySchema(BaseModel):
    image: ImageItem
    label: Optional[TxtNumpyItem] = None

    @validator("image")
    def check_image_size(cls, v):
        if v.shape[0] != 224 or v.shape[1] != 224:
            raise ValueError("Image must be 224x224")
        return v
```
Then:
```
{
    "schema": {
        "sample_schema": "my_schema.py:MySchema"
    }
}
```

**TIP**: to get a minimal schema for an existing dataset, try the following command:

In [ ]:
!pipelime validate +input.folder ../../tests/sample_data/datasets/underfolder_minimnist +max_samples 1

### Output Dataset And Serialization Modes

Like input datasets, an output dataset is mainly defined by the `folder` path and an
optional `schema` definition. Moreover, you can fine-tune how items are actually
serialized to disk.

The standard serialization procedure tries the following sequence of actions and stops
when one of them succeeds:
1. *remote file* (`REMOTE_FILE`): if remote source addresses are available, they are dumped
1. *hard link* (`HARD_LINK`): if file sources are available, one of them is hard linked
1. *deep copy* (`DEEP_COPY`): if file sources are available, one of them is copy
1. *new file* (`CREATE_NEW_FILE`): a new file is created by serializing the item value

Moreover, a *soft link* (`SYM_LINK`) option can be tried instead of *remote file* and
*hard link*, but only if **explicitly requested** (see below).

To alter this behavior, you can set the `serialization` option so as to override,
disable or force the desired mode, eg:
```
{
    "serialization": {
        "override": {
            "CREATE_NEW_FILE": ["PngImageItem", "BinaryItem"]
        }
        "disable": {
            "NumpyItem": "HARD_LINK"
        }
        "keys": {
            "mask": "REMOTE_FILE",
            "label": "SYM_LINK",
        }
    }
}
```

### The Grabber

The grabber interface provides a easy way to distribute data iteration on multiple
processes. The whole pipeline, included the item serialization, is executed in parallel,
but be aware that the process spawning overhead can be significant, so parameters should
be carefully tuned. The definition is as follows:
* `num_workers`: the number of processes to spawn. If negative, the number of (logical)
cpu cores is used.
* `prefetch`: the number of samples loaded in advanced by each worker, which might be
useful if the parent command has to post-process the data.